<img src="Images/HSP2.png" />
This Jupyter Notebook Copyright 2016 by RESPEC, INC.  All rights reserved.

$\textbf{HSP}^{\textbf{2}}\ \text{and}\ \textbf{HSP2}\ $ Copyright 2016 by RESPEC INC. and released under this [License](LegalInformation/License.txt)

# TUTORIAL 8: HSP2 with Design of Experiment (DoE)

## Motivation and concept notes
Frequently, $\textbf{HSP}^{\textbf{2}}$ is used for parameter sensitivity studies, parameter fits to data and optimizations. This requires HSP2 to  run many times (possibly thousands of runs for each study.)

The HSP2.runDoE() routine is an alternate main program for $\textbf{HSP}^{\textbf{2}}$designed to make this easy.

The normal HSP2.run() must be run first in order to ensure that the entire watershed has been run (at least once) and the resulting calculated timeseries are available.

Then the HSP2.runDoE() can be run which only recalculates the minimum necessary parts of the watershed. The arguments to **HSP2.runDoE(hdf5file, doe, measurements, prefix)** are:
+ **hd5file** is the name (with path if necessary) to the HDF5 file for the watershed,
+ **doe** is a Pandas DataFrame with a specific experimental design to run,
+ **measurements** (OPTIONAL) are a list of segment IDs were flow and/or water height data are available for comparison to the calculated results. If not specified, no assumptions about comparison data are made,
+ **prefix** (OPTIONAL) is a text label to be prepended to the run number. If not specified the default is "run".

The internal operations are:
+ Parse the doe DataFrame
  + Create a new temporary OP_SEQ table
      + determine the minimum set of calculations (a version of the *smart run* described in previous tutorials). 
      + If the optional measurements argument is specified, watershed segments below the final measurement segments are not run. That is, the *smart run* will run only the watershed segments starting from the segments defined in the doe DataFrame ending at the segments where the measurements are made.  Otherwise, the entire remaining watershed is run starting from the segments identified in the doe argument. The step redefines the OP_SEQUENCE table only for this run.

  + Create a dictionary (of dictionaries) to update the UCI at the proper time.
     + root dictionary has keys for each run specified in the DOE
     + each value in the root is a dictionary with keys: (operation, module) like (IMPLND, IWATER)
     + each value (next level) is a dictionary keyed by segment ID.
     + each value (next level) is a dictionary of parameter, value entries
+ For each run in the doe table:

    + Create a subdirectory named with the prefex followed by the run number. For example *run0001*.
    
    + Modify the specified parameters from run's doe data.
    
    + run the minimal (*smart run*) watershed
    
    + save calculated results in the subdirectory under a subdirectory named *results*.
        + insure that time series reads pick from the correct directory (main or current run)!
    + Update the HDF5 key list?  **NOTE:**  store.keys() is still slow -- 8.7 seconds for calleg.h5 and 0.5 seconds for test10.
    
When all runs are complete, a post run analysis can be used to determine the desired sensitivity analysis, optimal parameter fits, or watershed management optimizations.

### Required Python imports  and setup 

In [ ]:
import os
import site
site.addsitedir(os.getcwd().rsplit('\\',1)[0] + '\\')  # adds your path to the HSP2 software.

import numpy as np
import pandas as pd
pd.options.display.max_rows    = 18
pd.options.display.max_columns = 20
pd.options.display.float_format = '{:.3f}'.format  # display 2 digits after the decimal point

import matplotlib.pyplot as plt
%matplotlib inline

import HSP2
import HSP2tools

hdfname = os.path.join('TutorialData', 'Tutorial.h5')

HSP2tools.reset_tutorial()    # make a new copy of the tutorial's data
HSP2tools.versions()          # display version information below

##  Design of Experiment plans

There are many sources for Design of Experiment plans. It is easy to take a DoE plan and add the additional data required by $\textbf{HSP}^{\textbf{2}}$  as a Pandas DataFrame.

The $\textbf{HSP}^{\textbf{2}}$ DoE DataFrame has these parameter (in this order):
+ **run** ID needed to associate the parameter values to the computed timeseries,
+ **operation** (PERLND, IMPLND, RCHRES), 
+ **segment** ID for the parameter being varied in this row (pid, iid, rid),
+ **module** name containing the parameter,
+ **parameter** name,
+ **value** to be assigned to this parameter in this run.

If the same parameter is being modified in several segments, just include a row for each segment.

There is no limit to the number of rows in the DataFrame.

## Sensitivity Studies
To speed Design of Experiment Studies, only the segments impacted by  the DoE variables will be run, "SMART OP_SEQUENCE".  So it is necessary to first do a complete normal run.

In [ ]:
HSP2.run(hdfname)  # needed one time to get all necessary output available.

This will be rerun (not necessary) to show the timing once the initial JIT (Just In Time) compilation has been done.

In [ ]:
HSP2.run(hdfname) 

Typical Design of Experiment plans for sensitivity study will vary all of a group of parameters one at a time by the same percentage.

Analysis is simple. Pick a quantity of interest (QOI for you  statistians) from the computed timeseries and rank the variation be the magnitude of the change. Generally, only a few parameters will demonstrate significantly larger variation compared to the rest.

**NOTE**  This type study will not find interactions between parameters. Normally, this type of study is used to eliminate parameters that seem to provide only minor variation. Then a DoE plan which looks at interactions will be  run for the significant parameters. Better design of experiment plans are available.

The following cell will build a simple sensitivity DoE plan for the IMPLND segment in Test 10. Each parameter is being provided at +/- 20 percent of its normal value.

In [ ]:
data = [
 ['1', 'IMPLND', 'I001', 'IWATER', 'LSUR',    200 * 0.8],
 ['2', 'IMPLND', 'I001', 'IWATER', 'LSUR',    200 * 1.2],
 ['3', 'IMPLND', 'I001', 'IWATER', 'NSUR',   0.01 * 0.8],
 ['4', 'IMPLND', 'I001', 'IWATER', 'NSUR',   0.01 * 1.2],
 ['5', 'IMPLND', 'I001', 'IWATER', 'PETMAX', 40.0 * 0.8],
 ['6', 'IMPLND', 'I001', 'IWATER', 'PETMAX', 40.0 * 1.2], 
 ['7', 'IMPLND', 'I001', 'IWATER', 'PETMIN', 35.0 * 0.8],
 ['8', 'IMPLND', 'I001', 'IWATER', 'PETMIN', 35.0 * 1.2],
 ['9', 'IMPLND', 'I001', 'IWATER', 'RETS',   0.01 * 0.8],
 ['10','IMPLND', 'I001', 'IWATER', 'RETS',   0.01 * 1.2],
 ['11','IMPLND', 'I001', 'IWATER', 'RETSC',  0.01 * 0.8],
 ['12','IMPLND', 'I001', 'IWATER', 'RETSC',  0.01 * 1.2],
 ['13','IMPLND', 'I001', 'IWATER', 'SLSUR',  0.01 * 0.8],
 ['14','IMPLND', 'I001', 'IWATER', 'SLSUR',  0.01 * 1.2],
 ['15','IMPLND', 'I001', 'IWATER', 'SURS',   0.01 * 0.8],
 ['16','IMPLND', 'I001', 'IWATER', 'SURS',   0.01 * 1.2]]

doe = pd.DataFrame(data, columns=['Run', 'Operation', 'Segment', 'Module', 'Parameter', 'Value'])
doe

#### Run this DoE plan

In [ ]:
HSP2.run_DoE(hdfname, 'ImplndSensitivity', doe)

**Note:** From the timing printed with each run, this is much faster than running HSP2 16 times (one per parameter set change.) For real use, the experimental design will require 100 to 1000 runs.  This can be a significant runtime reduction.

**Note:**  It is useful to use HDFView or HDFCompass to look at the resulting HDF5 file.

## Optimal Parameter Fitting

There are many DoE methods for performing parameter fits. This example will use a simple Monte Carlo method to randomly create a value for each parameter of interest drawn from normal distributions specific to each parameter.

Monte Carlo methods take a lot of runs, but are very simple to create. Other methods to consider are *hill climbing, genetic algorithms, Latin Hypercube sampling, and orthogonal arrays.*

In this example, a random value from a normal distribution is created using this numpy idiom:
```
randompick = mu + sigma * np.random.rand(1)
```
where the result is a pick from n(mu, sigma) with
+ mu is the mean
+ sigma is the standard deviation

Since this will return a one element numpy array, the float() conversion is used to return a single value.

Three parameters were (arbitrarily) picked  to be fitted simultaneously.

**NOTE**  Good statistical practice is to vary all selected parameters in each run.  The change of one parameter at a time DOES NOT provide good results and yet takes more runs than a good experimental design.

The following code will create the necessary DataFrame. The value of N is selected to be small only to make a fast running example.

In [ ]:
N = 10   # number of runs - generally large such as 1000 or 10,000

data = []
for r in range(1, N):
    run = str(r)
    data.append([run,'IMPLND','I001','IWATER','LSUR', 200.0 + float(np.random.rand(1)) * 15])
    data.append([run,'IMPLND','I001','IWATER','NSUR', 0.01  + float(np.random.rand(1)) * 0.005])
    data.append([run,'IMPLND','I001','IWATER','RETS', 0.01  + float(np.random.rand(1)) * 0.003])
   
doe = pd.DataFrame(data, columns=['Run', 'Operation', 'Segment', 'Module', 'Parameter', 'Value'])
doe

#### Run this DoE plan

In [ ]:
HSP2.run_DoE(hdfname, 'ImplndMCfit1', doe)

Analysis is fairly straight forward.  At one or more locations where measurement data are available (such as flow rate or water level), take the computed timeseries for a run at that location and compute a comparison metric between the measurement timeseries to the simulation timeseries. (You might need to aggregate metrics to match multiple locations simultaneously.)  Typically, the comparison might be least square metric between two timeseries.

Rank the metrics to find the closest overall match, and set the parameters to the values used in best run.

If more accuracy is needed (perhaps the match wasn't good enough), then use a few of the top ranked 
parameter sets as starting points each with tight standard deviations around its values in another Monte Carlo simulation(s).